In [1]:
import tkinter as tk
from tkinter import messagebox, simpledialog
import os

class ContactManager:
    def __init__(self, filename="friendsContact.txt"):
        self.filename = filename
        existe_archivo = os.path.exists(self.filename)
        if not existe_archivo:
            archivo = open(self.filename, "w")
            archivo.close()

    def create_contact(self, name, number):
        if name != "" and number != "":
            archivo = open(self.filename, "a")
            linea = name + "," + number + "\n"
            archivo.write(linea)
            archivo.close()
            return "✓ Contacto creado"
        else:
            return "Debe ingresar nombre y número"

    def read_contacts(self):
        try:
            archivo = open(self.filename, "r")
            lineas = archivo.readlines()
            archivo.close()
            lista_contactos = []
            for linea in lineas:
                partes = linea.strip().split(",")
                lista_contactos.append(partes)
            return lista_contactos
        except:
            return []

    def search_contact(self, name):
        contactos = self.read_contacts()
        for contacto in contactos:
            if contacto[0].lower() == name.lower():
                return contacto[1]
        return None

    def update_contact(self, name, new_number):
        contactos = self.read_contacts()
        nuevo_contenido = []
        encontrado = False
        for contacto in contactos:
            if contacto[0].lower() == name.lower():
                nuevo_contenido.append([contacto[0], new_number])
                encontrado = True
            else:
                nuevo_contenido.append(contacto)
        archivo = open(self.filename, "w")
        for contacto in nuevo_contenido:
            linea = contacto[0] + "," + contacto[1] + "\n"
            archivo.write(linea)
        archivo.close()
        if encontrado:
            return "✓ Contacto actualizado"
        else:
            return "Contacto no encontrado"

    def delete_contact(self, name):
        contactos = self.read_contacts()
        nuevos = []
        borrado = False
        for contacto in contactos:
            if contacto[0].lower() != name.lower():
                nuevos.append(contacto)
            else:
                borrado = True
        archivo = open(self.filename, "w")
        for contacto in nuevos:
            archivo.write(contacto[0] + "," + contacto[1] + "\n")
        archivo.close()
        if borrado:
            return "✓ Contacto eliminado"
        else:
            return "Contacto no encontrado"

class ContactApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Gestor de Contactos")
        self.manager = ContactManager()
        self.root.geometry("400x400")

        bg_color = "#f0f8ff"
        button_color = "#4682b4"
        text_color = "#2f4f4f"

        root.configure(bg=bg_color)

        main_frame = tk.Frame(root, bg=bg_color)
        main_frame.pack(padx=20, pady=20, fill=tk.BOTH, expand=True)

        tk.Label(main_frame, text="Nombre:", bg=bg_color, fg=text_color, font=("Arial", 10, "bold")).grid(row=0, column=0, padx=5, pady=5, sticky="e")
        self.name_entry = tk.Entry(main_frame, width=25, font=("Arial", 10))
        self.name_entry.grid(row=0, column=1, padx=5, pady=5)

        tk.Label(main_frame, text="Número:", bg=bg_color, fg=text_color, font=("Arial", 10, "bold")).grid(row=1, column=0, padx=5, pady=5, sticky="e")
        self.number_entry = tk.Entry(main_frame, width=25, font=("Arial", 10))
        self.number_entry.grid(row=1, column=1, padx=5, pady=5)

        button_config = {
            'bg': button_color,
            'fg': 'white',
            'width': 12,
            'height': 1,
            'font': ("Arial", 9, "bold")
        }

        buttons_frame = tk.Frame(main_frame, bg=bg_color)
        buttons_frame.grid(row=2, column=0, columnspan=2, pady=15)

        tk.Button(buttons_frame, text="CREAR", command=self.create, **button_config).pack(side=tk.LEFT, padx=5)
        tk.Button(buttons_frame, text="LEER", command=self.read, **button_config).pack(side=tk.LEFT, padx=5)
        tk.Button(buttons_frame, text="ACTUALIZAR", command=self.update, **button_config).pack(side=tk.LEFT, padx=5)
        tk.Button(buttons_frame, text="ELIMINAR", command=self.delete, **button_config).pack(side=tk.LEFT, padx=5)

        results_frame = tk.LabelFrame(main_frame, text=" Resultados ", bg=bg_color, fg=text_color, font=("Arial", 10, "bold"))
        results_frame.grid(row=3, column=0, columnspan=2, padx=5, pady=10, sticky="nsew")

        self.text_area = tk.Text(results_frame, width=45, height=10, bg="white", fg=text_color, font=("Arial", 10), wrap=tk.WORD)
        self.text_area.pack(padx=10, pady=10)
        self.text_area.insert(tk.END, "➤ Ingrese un nombre y haga clic en LEER para buscar un contacto. Deje vacío para ver todos.")
        self.text_area.config(state=tk.DISABLED)

    def create(self):
        name = self.name_entry.get().strip()
        number = self.number_entry.get().strip()
        resultado = self.manager.create_contact(name, number)
        messagebox.showinfo("Resultado", resultado)
        self.name_entry.delete(0, tk.END)
        self.number_entry.delete(0, tk.END)

    def read(self):
        name = self.name_entry.get().strip()
        self.text_area.config(state=tk.NORMAL)
        self.text_area.delete(1.0, tk.END)
        if name != "":
            number = self.manager.search_contact(name)
            if number != None:
                self.text_area.insert(tk.END, "Contacto encontrado:\n\n")
                self.text_area.insert(tk.END, "• Nombre: " + name + "\n")
                self.text_area.insert(tk.END, "• Número: " + number)
            else:
                self.text_area.insert(tk.END, "✗ No se encontró el contacto: " + name)
        else:
            contactos = self.manager.read_contacts()
            if len(contactos) > 0:
                self.text_area.insert(tk.END, "📚 Lista de contactos:\n\n")
                for contacto in contactos:
                    self.text_area.insert(tk.END, "• " + contacto[0] + ": " + contacto[1] + "\n")
            else:
                self.text_area.insert(tk.END, "➤ No hay contactos registrados")
        self.text_area.config(state=tk.DISABLED)

    def update(self):
        name = self.name_entry.get().strip()
        if name == "":
            messagebox.showerror("Error", "Ingrese nombre a actualizar")
            return
        new_number = simpledialog.askstring("Actualizar", "Nuevo número para " + name + ":")
        if new_number:
            resultado = self.manager.update_contact(name, new_number)
            messagebox.showinfo("Resultado", resultado)
            self.read()

    def delete(self):
        name = self.name_entry.get().strip()
        if name == "":
            messagebox.showerror("Error", "Ingrese nombre a eliminar")
            return
        resultado = self.manager.delete_contact(name)
        messagebox.showinfo("Resultado", resultado)
        self.read()

if __name__ == "__main__":
    root = tk.Tk()
    app = ContactApp(root)
    root.mainloop()
